# Personalization Agent Demo

## Connect to the Weaviate Cloud instance

> Reminder: Weaviate Agents are only available for Weaviate Cloud instances.

Connect to your Weaviate instance, using credentials from the Weaviate Cloud console. Here, they are loaded from the `.env` file.

In [1]:
from dotenv import load_dotenv
import weaviate
import os

load_dotenv()

weaviate_url = os.getenv("WEAVIATE_URL")
weaviate_api_key = os.getenv("WEAVIATE_API_KEY")

client = weaviate.connect_to_weaviate_cloud(
    cluster_url=weaviate_url,
    auth_credentials=weaviate_api_key,
)

assert client.is_ready()

## Connect to the Personalization Agent

You can initialize the Personalization Agent, or connect to an existing one, as shown below.

In [2]:
from weaviate.agents.personalization import PersonalizationAgent
from weaviate.classes.config import DataType

collection_name = "Movie"

if PersonalizationAgent.exists(client, collection_name):
    pa = PersonalizationAgent.connect(
        client=client,
        reference_collection=collection_name,
        vector_name="default",
    )
else:
    pa = PersonalizationAgent.create(
        client=client,
        reference_collection=collection_name,
        vector_name="default",
        user_properties={
            "age": DataType.NUMBER,
            "favorite_genres": DataType.TEXT_ARRAY,
            "favorite_years": DataType.NUMBER_ARRAY,
            "language": DataType.TEXT,
        },
    )

## Create a persona

A "persona" is where the agent stores its knowledge about a user. You can add a persona or use an existing one. 

In [3]:
from weaviate.agents.classes import Persona
from weaviate.util import generate_uuid5
from uuid import uuid4  # If you want to generate a random UUID

persona_id = generate_uuid5("jphwang")  # To generate a deterministic UUID
# persona_id = uuid4()  # To generate a random UUID

# You can delete a persona if you want to remove it from the system
pa.delete_persona(persona_id)

if pa.has_persona(persona_id):
    print(f"Persona with ID {persona_id} already exists.")
else:
    print(f"Creating new persona with ID {persona_id}.")
    pa.add_persona(
        Persona(
            persona_id=persona_id,
            properties={
                "age": 18,
                "favorite_genres": ["Sci-Fi", "Fantasy", "Action"],
                "favorite_years": [1999, 2008, 2018, 2019],
                "language": "English",
            },
        )
    )

Creating new persona with ID 2fb2f796-8f16-5dc9-8e5d-c285a92a9dac.


## Add interactions

This is how the agent learns each persona's preferences.

In [ ]:
from weaviate.agents.classes import PersonaInteraction
from helpers import get_movie_uuid  # Helper to get the UUID of a movie

pa.add_interactions(interactions=[
    PersonaInteraction(
        persona_id=persona_id, item_id=get_movie_uuid(client, "Independence Day"), weight=0.8
    ),
])

Fetched movie 'Independence Day' from the collection


## Queries

We can already perform queries.

### Basic queries

Fast, most basic personalized queries

- Uses vectors of interaction history only

In [5]:
response = pa.get_objects(persona_id, limit=50, use_agent_ranking=False)

In [6]:
from helpers import print_movie_response_details

print_movie_response_details(response, 5)

*****0*****
Independence Daysaster
None
original rank: 0, personalized rank: None
*****1*****
Independence Day
None
original rank: 1, personalized rank: None
*****2*****
Alien Resurrection
None
original rank: 2, personalized rank: None
*****3*****
Star Trek: Insurrection
None
original rank: 3, personalized rank: None
*****4*****
Redemption Day
None
original rank: 4, personalized rank: None


### Agent reranking

The agent can smartly rerank the results based on the information about the persona, as well as the interactions.

In [7]:
response = pa.get_objects(persona_id, limit=50, use_agent_ranking=True)

print_movie_response_details(response, 5)

Ranking rationale: You love sci-fi, fantasy, and action movies, especially those involving alien invasions and high-stakes battles. We've prioritized titles that strongly reflect these genres, have high engagement, and fall into your favorite years or are related to iconic sci-fi franchises. This list balances well-known hits with exciting action-packed and imaginative adventures that match your youthful, dynamic taste.
*****0*****
Independence Day
None
original rank: 1, personalized rank: 0
*****1*****
Prometheus
None
original rank: 37, personalized rank: 1
*****2*****
Occupation
None
original rank: 9, personalized rank: 2
*****3*****
Pacific Rim: Uprising
None
original rank: 19, personalized rank: 3
*****4*****
Serenity
None
original rank: 15, personalized rank: 4


### With Reranker + Instruction

- Uses vectors of interaction history and AI-based reranker
- Instructions used to guide the reranker

In [8]:
response = pa.get_objects(
    persona_id,
    limit=50,
    use_agent_ranking=True,
    instruction="I'm looking for something for the whole family, maybe a fun, light action film."
)

print_movie_response_details(response, 5)

Ranking rationale: Since you're looking for a fun, light action film suitable for the whole family and you enjoy genres like Sci-Fi, Fantasy, and Action from specific years, I've prioritized movies that balance family-friendly vibes with exciting action and sci-fi/fantasy elements. Titles with a lighter tone, family appeal, and strong action or fantasy themes have been boosted, while darker, horror-heavy, or more intense adult-oriented entries have been ranked lower.
*****0*****
Rise of the Guardians
None
original rank: 49, personalized rank: 1
*****1*****
Pacific Rim: Uprising
None
original rank: 19, personalized rank: 2
*****2*****
The Matrix Resurrections
None
original rank: 41, personalized rank: 3
*****3*****
Independence Day
None
original rank: 1, personalized rank: 4
*****4*****
Independence Daysaster
None
original rank: 0, personalized rank: 5


### Add more interactions

Over time, you will add more interactions to the agent, which will help it learn more about the persona's preferences.

Note each interaction can be positive or negative. 
(1: most positive, 0: neutral, -1: most negative)

In [ ]:
interactions = [
    PersonaInteraction(
        persona_id=persona_id, item_id=get_movie_uuid(client, "Iron Man"), weight=0.9  # very positive
    ),
    PersonaInteraction(
        persona_id=persona_id, item_id=get_movie_uuid(client, "The Grand Budapest Hotel"), weight=0.9
    ),
    PersonaInteraction(
        persona_id=persona_id, item_id=get_movie_uuid(client, "Sleepless in Seattle"), weight=0.8
    ),
    PersonaInteraction(
        persona_id=persona_id, item_id=get_movie_uuid(client, "The Mummy"), weight=0.0  # neutral
    ),
    PersonaInteraction(
        persona_id=persona_id, item_id=get_movie_uuid(client, "A Nightmare on Elm Street"), weight=-0.3,  # little bit negative
    ),
    PersonaInteraction(
        persona_id=persona_id, item_id=get_movie_uuid(client, "The Cloverfield Paradox"), weight=-0.9  # very negative
    ),
]

pa.add_interactions(interactions=interactions)

Fetched movie 'Iron Man' from the collection
Fetched movie 'The Grand Budapest Hotel' from the collection
Fetched movie 'Sleepless in Seattle' from the collection
Fetched movie 'The Mummy' from the collection
Fetched movie 'A Nightmare on Elm Street' from the collection
Fetched movie 'The Cloverfield Paradox' from the collection


### Retry with the updated knowledge

In [10]:
response = pa.get_objects(persona_id, limit=50, use_agent_ranking=True)

print_movie_response_details(response, 5)

Ranking rationale: Since you enjoy Sci-Fi, Fantasy, and Action movies especially from around 1999, 2008, and recent years, we've prioritized films featuring superheroes, alien invasions, and adventurous quests. Classics like Iron Man and Avengers movies scored high due to their genre, popularity, and your positive interest in similar titles. We've also placed higher-ranked movies with strong action and science fiction elements, while minimizing less relevant genres or lower-rated films.
*****0*****
Iron Man
None
original rank: 4, personalized rank: 1
*****1*****
Iron Man 3
None
original rank: 6, personalized rank: 2
*****2*****
Avengers: Age of Ultron
None
original rank: 38, personalized rank: 3
*****3*****
The Invincible Iron Man
None
original rank: 2, personalized rank: 4
*****4*****
Serenity
None
original rank: 24, personalized rank: 5


### With Reranker + Instruction + Filter

- The most complex personalized queries
- Uses vectors of interaction history and AI-based reranker
- Instructions used to guide the reranker
- Filters out items that are not relevant to the user

In [14]:
# With Reranker + Instruction + Filter
response = pa.get_objects(
    persona_id,
    limit=50,
    use_agent_ranking=True,
    instruction="The user is looking for a classic drama, that is suitable for a date night.",
    filters=Filter.by_property("release_year").less_or_equal(2000)
)

In [15]:
print_movie_response_details(response, 10)

Ranking rationale: Since you're looking for a classic drama suitable for a date night, I've prioritized timeless romantic dramas and classics with strong emotional storytelling. While you favor sci-fi and action generally, this recommendation focuses on more intimate and romantic narratives to suit your date night mood.
*****0*****
The Apartment
None
original rank: 41, personalized rank: 1
*****1*****
The Graduate
None
original rank: 9, personalized rank: 2
*****2*****
Manhattan
None
original rank: 1, personalized rank: 3
*****3*****
Roman Holiday
None
original rank: 39, personalized rank: 4
*****4*****
Great Expectations
None
original rank: 33, personalized rank: 5
*****5*****
The Great Gatsby
None
original rank: 35, personalized rank: 6
*****6*****
Tess
None
original rank: 25, personalized rank: 7
*****7*****
The Night Porter
None
original rank: 29, personalized rank: 8
*****8*****
Emmanuelle
None
original rank: 15, personalized rank: 9
*****9*****
Naked
None
original rank: 45, perso

## Combine personalization with other queries

From `pa.query`, you can perform the common Weaviate searches, such as `near_text`, `bm25` and `hybrid`

In [16]:
response = pa.query(persona_id=persona_id, strength=0.95).hybrid(
    query="historical adventure",
    limit=10
)

for o in response.objects:
    print(f"Title: {o.properties['title']}")
    print(f"Genres: {o.properties['genres']}")

Title: The Adventures of Baron Munchausen
Genres: None
Title: The Adventures of Robin Hood
Genres: None
Title: Timeline
Genres: None
Title: Bill & Ted's Excellent Adventure
Genres: None
Title: Firewalker
Genres: None
Title: The Extraordinary Adventures of Adèle Blanc-Sec
Genres: None
Title: Jack Hunter and the Quest for Akhenaten's Tomb
Genres: None
Title: Pilgrimage
Genres: None
Title: Hidalgo
Genres: None
Title: King Arthur
Genres: None


In [18]:
client.close()